In [62]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [63]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [64]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['course']}-{doc['question']}-{doc['text'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

for doc in documents:
    doc['id'] = generate_document_id(doc)

In [65]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'id': 'c02e79ef'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'id': '1f6520ca'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware

In [66]:
import json

with open('documents-with-ids.json', 'wt') as f_out:
    json.dump(documents, f_out, indent=2)

In [67]:
!head documents-with-ids.json

[
  {
    "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
    "section": "General course-related questions",
    "question": "Course - When will the course start?",
    "course": "data-engineering-zoomcamp",
    "id": "c02e79ef"
  },
  {
    "text": "GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites",


In [68]:
prompt_template = """
You emulate a student who is taking our course.
Formulate 5 questions this student might ask based on a faq record. The 
record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer word as possible from the record.

The record: 

section: {section}
question: {question}
answer: {text}

Provide the output in parsable json without using code blocks:

["question1", "question2",....., "question5"]
""".strip()
print(prompt_template)

You emulate a student who is taking our course.
Formulate 5 questions this student might ask based on a faq record. The 
record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer word as possible from the record.

The record: 

section: {section}
question: {question}
answer: {text}

Provide the output in parsable json without using code blocks:

["question1", "question2",....., "question5"]


In [69]:
from mistralai import Mistral

model = "mistral-small"
api_key = 'mM6V2SsnI9IzQXWcn4Fr8xCKYt11HFiT'
client = Mistral(api_key=api_key)

In [70]:
doc = documents[2]
prompt = prompt_template.format(**doc)
print(prompt)

You emulate a student who is taking our course.
Formulate 5 questions this student might ask based on a faq record. The 
record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer word as possible from the record.

The record: 

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Provide the output in parsable json without using code blocks:

["question1", "question2",....., "question5"]


In [71]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)
    response = client.chat.complete(
        model=model,
        messages=[{
            "role": "user",
            "content": prompt
        }]
    )
    json_response = response.choices[0].message.content    
    return json_response

In [72]:
print(generate_questions(doc))

[
"Is it possible to sign up for the course after the start date?",
"If I can't join the course at the beginning, will I still be able to submit assignments?",
"Are there deadlines for submitting final projects in the course?",
"What happens if I don't register for the course at the start date?",
"Should I worry about completing the final projects last minute in this course?"]


In [82]:
from tqdm.auto import tqdm

results = {}

for doc_item in tqdm(documents[:10]):
    doc_id = doc_item['id']
    if doc_id in results:
        continue 

    questions = generate_questions(doc_item)
    results[doc_id] = questions


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.98s/it]


In [101]:
with open('results.bin', 'rb') as f:
    results = pickle.load(f)

In [108]:
parsed_result = {}

for doc_id, json_questions in results.items():
    parsed_result[doc_id] = json.loads(json_questions)

In [105]:
json_questions = [
r"How can I resolve the Docker error 'invalid mode: \Program Files\Git\var\lib\postgresql\data'?",
"What should I do if I encounter an invalid mode error in Docker on Windows?",
"What is the correct mounting path to use in Docker for PostgreSQL data on Windows?",
"Can you provide an example of a correct Docker mounting path for PostgreSQL data?",
r"How do I correct the mounting path error in Docker for \Program Files\Git\var\lib\postgresql\data'?"
]

In [106]:
results[doc_id] = json.dumps(json_questions)

In [131]:
doc_index = {d['id']: d for d in documents} # question - course - id
doc_index['c02e79ef']

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [130]:
final_result = []
for doc_id, questions in parsed_result.items():
    course = doc_index[doc_id]['course']
    for q in questionsq
        final_result.append((q, course, doc_id))

In [135]:
import pandas as pd 

df = pd.DataFrame(final_result, columns=["question", "course", "document"])

In [136]:
df.to_csv("Data.csv")

In [145]:
df_data = pd.read_csv('Data.csv')
df_data

,Unnamed: 0,question,course,document
0,0,When does the course begin?,data-engineering-zoomcamp,c02e79ef
1,1,How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
2,2,What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
3,3,How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
4,4,Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
...,...,...,...,...
4622,4622,How should I destroy infrastructure created us...,mlops-zoomcamp,886d1617
4623,4623,What is the first step to destroy AWS infrastr...,mlops-zoomcamp,886d1617
4624,4624,Can I destroy infrastructure created with GitH...,mlops-zoomcamp,886d1617
4625,4625,What command initializes Terraform with specif...,mlops-zoomcamp,886d1617
